In [1]:
!pip install mlflow boto3 awscli optuna imbalanced-learn lightgbm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.8/25.8 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 70.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.1/140.1 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 84.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 82.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 570.5/570.5 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.

In [ ]:
!aws configure

In [3]:
import mlflow

mlflow.set_tracking_uri(f'{userdata.get('MLFLOW_SERVER')}')

In [ ]:
mlflow.set_experiment("Light GBM HP Tuning")

In [5]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import mlflow
import mlflow.sklearn
import optuna
from lightgbm import LGBMClassifier
import matplotlib.pyplot as plt

In [9]:
df = pd.read_csv("/content/reddit_preprocessing.csv")
df.dropna(inplace=True)
df.shape

(36662, 2)

In [10]:
# Step 1: Remap the class labels from [-1, 0, 1] to [2, 0, 1]
df['category'] = df['category'].map({-1: 2, 0: 0, 1: 1})

# Step 2: Remove rows where the target labels (category) are NaN
df = df.dropna(subset=['category'])

In [11]:
ngram_range = (1, 3)  # Trigram
max_features = 1000  # Set max_features to 1000
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X = vectorizer.fit_transform(df['clean_comment'])
y = df['category']

# Step 4: Apply SMOTE to handle class imbalance
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)
# Step 5: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled)

In [7]:
# Function to log results in MLflow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test, params, trial_number):
    with mlflow.start_run():
        # Log model type and trial number
        mlflow.set_tag("mlflow.runName", f"Trial_{trial_number}_{model_name}_SMOTE_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Log hyperparameters
        for key, value in params.items():
            mlflow.log_param(key, value)

        # Train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")

        return accuracy



In [8]:
# Step 6: Optuna objective function for LightGBM
def objective_lightgbm(trial):
    # Hyperparameter space to explore
    n_estimators = trial.suggest_int('n_estimators', 100, 1000)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True)
    max_depth = trial.suggest_int('max_depth', 3, 15)
    num_leaves = trial.suggest_int('num_leaves', 20, 150)
    min_child_samples = trial.suggest_int('min_child_samples', 10, 100)
    colsample_bytree = trial.suggest_float('colsample_bytree', 0.5, 1.0)
    subsample = trial.suggest_float('subsample', 0.5, 1.0)
    reg_alpha = trial.suggest_float('reg_alpha', 1e-4, 10.0, log=True)  # L1 regularization
    reg_lambda = trial.suggest_float('reg_lambda', 1e-4, 10.0, log=True)  # L2 regularization

    # Log trial parameters
    params = {
        'n_estimators': n_estimators,
        'learning_rate': learning_rate,
        'max_depth': max_depth,
        'num_leaves': num_leaves,
        'min_child_samples': min_child_samples,
        'colsample_bytree': colsample_bytree,
        'subsample': subsample,
        'reg_alpha': reg_alpha,
        'reg_lambda': reg_lambda
    }

    # Create LightGBM model
    model = LGBMClassifier(n_estimators=n_estimators,
                           learning_rate=learning_rate,
                           max_depth=max_depth,
                           num_leaves=num_leaves,
                           min_child_samples=min_child_samples,
                           colsample_bytree=colsample_bytree,
                           subsample=subsample,
                           reg_alpha=reg_alpha,
                           reg_lambda=reg_lambda,
                           random_state=42,
                           verbose=-1,
                        force_row_wise=True  )

    # Log each trial as a separate run in MLflow
    accuracy = log_mlflow("LightGBM", model, X_train, X_test, y_train, y_test, params, trial.number)

    return accuracy

In [12]:
# Step 7: Run Optuna for LightGBM, log the best model, and plot the importance of each parameter
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_lightgbm, n_trials=100)  # Increased to 100 trials

    # Get the best parameters
    best_params = study.best_params
    best_model = LGBMClassifier(n_estimators=best_params['n_estimators'],
                                learning_rate=best_params['learning_rate'],
                                max_depth=best_params['max_depth'],
                                num_leaves=best_params['num_leaves'],
                                min_child_samples=best_params['min_child_samples'],
                                colsample_bytree=best_params['colsample_bytree'],
                                subsample=best_params['subsample'],
                                reg_alpha=best_params['reg_alpha'],
                                reg_lambda=best_params['reg_lambda'],
                                random_state=42)

    # Log the best model with MLflow and print the classification report
    log_mlflow("LightGBM", best_model, X_train, X_test, y_train, y_test, best_params, "Best")

    # Plot parameter importance
    optuna.visualization.plot_param_importances(study).show()

    # Plot optimization history
    optuna.visualization.plot_optimization_history(study).show()

In [13]:
run_optuna_experiment()

[I 2025-08-10 15:49:03,488] A new study created in memory with name: no-name-76545768-1cf4-45cf-91e7-bf17c9ca4f09
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
2025/08/10 15:50:35 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/10 15:50:46 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
[I 2025-08-10 15:50:47,736] Trial 0 finished with value: 0.8191714225322342 and parameters: {'n_estimators': 916, 'learning_rate': 0.05129428805931973, 'max_depth': 11, 'num_leaves': 40, 'min_child_samples': 11, 'colsample_bytree': 0.8934042756399512, 'subsample': 0.8147456916619786, 'reg_alpha': 0.031942907854620825, 'reg_lambda': 8.999450296335478}. Best is trial 0 with value: 0.819171422

🏃 View run Trial_0_LightGBM_SMOTE_TFIDF_Trigrams at: http://ec2-52-91-229-41.compute-1.amazonaws.com:5000/#/experiments/639329270342233827/runs/bbefa246baec491c81646308202a914e
🧪 View experiment at: http://ec2-52-91-229-41.compute-1.amazonaws.com:5000/#/experiments/639329270342233827


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
2025/08/10 15:51:08 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/10 15:51:14 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
[I 2025-08-10 15:51:14,758] Trial 1 finished with value: 0.6679348974846755 and parameters: {'n_estimators': 167, 'learning_rate': 0.00022855864330871025, 'max_depth': 10, 'num_leaves': 143, 'min_child_samples': 21, 'colsample_bytree': 0.6885753309981542, 'subsample': 0.9390393584048917, 'reg_alpha': 0.6626329019001337, 'reg_lambda': 0.0023466830781409065}. Best is trial 0 with value: 0.8191714225322342.


🏃 View run Trial_1_LightGBM_SMOTE_TFIDF_Trigrams at: http://ec2-52-91-229-41.compute-1.amazonaws.com:5000/#/experiments/639329270342233827/runs/75dcf2550166403592dff54c42813094
🧪 View experiment at: http://ec2-52-91-229-41.compute-1.amazonaws.com:5000/#/experiments/639329270342233827


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
2025/08/10 15:52:00 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/10 15:52:06 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
[I 2025-08-10 15:52:07,160] Trial 2 finished with value: 0.6627562883111393 and parameters: {'n_estimators': 537, 'learning_rate': 0.004635262821262116, 'max_depth': 4, 'num_leaves': 24, 'min_child_samples': 85, 'colsample_bytree': 0.5693541018855066, 'subsample': 0.7379589512106857, 'reg_alpha': 0.030590962405267864, 'reg_lambda': 0.01035144108232426}. Best is trial 0 with value: 0.8191714225322342.


...

🏃 View run Trial_98_LightGBM_SMOTE_TFIDF_Trigrams at: http://ec2-52-91-229-41.compute-1.amazonaws.com:5000/#/experiments/639329270342233827/runs/f9cc2eddc716464387a1c4d9e30795ad
🧪 View experiment at: http://ec2-52-91-229-41.compute-1.amazonaws.com:5000/#/experiments/639329270342233827


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
2025/08/10 17:56:53 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/10 17:56:59 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
[I 2025-08-10 17:57:00,292] Trial 99 finished with value: 0.816423589093215 and parameters: {'n_estimators': 573, 'learning_rate': 0.041893319540390245, 'max_depth': 13, 'num_leaves': 34, 'min_child_samples': 24, 'colsample_bytree': 0.6534603070208127, 'subsample': 0.9519053660585197, 'reg_alpha': 0.030692988511974693, 'reg_lambda': 0.033498024991901514}. Best is trial 72 with value: 0.8245614035087719.


🏃 View run Trial_99_LightGBM_SMOTE_TFIDF_Trigrams at: http://ec2-52-91-229-41.compute-1.amazonaws.com:5000/#/experiments/639329270342233827/runs/41c7059b9f4045b9a188460f97b03a68
🧪 View experiment at: http://ec2-52-91-229-41.compute-1.amazonaws.com:5000/#/experiments/639329270342233827


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
2025/08/10 17:58:12 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/10 17:58:18 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Trial_Best_LightGBM_SMOTE_TFIDF_Trigrams at: http://ec2-52-91-229-41.compute-1.amazonaws.com:5000/#/experiments/639329270342233827/runs/8861175d4d5b44d583da700a1bd1a202
🧪 View experiment at: http://ec2-52-91-229-41.compute-1.amazonaws.com:5000/#/experiments/639329270342233827
